# Diabetes Patients Early Readmissions Prediction

**Authors:** Peter Macinec, Frantisek Sefcik

## Data Preprocessing

In this jupyter notebook, we will try to preprocess the data, create functions for preprocessing and define preprocessing pipeline.

### Setup and import libraries

# TODO

- drop attributes with too many values (or just group smallest classes)
- encoding values (one-hot)
- oversampling/undersampling
- outliers removal?
- categorical with too few values into "other"?
- age can be converted to average
- parts from feature engineering


- copy into file and one preprocessing function

In [1]:
# Automatically reloading imported modules
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
sys.path.append('..')

from sklearn.pipeline import Pipeline
from src.preprocessing.transformers import *

In [3]:
df = pd.read_csv('../data/data.csv', na_values='?', low_memory=False)

In [4]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


### Preprocessing

TODO - description about building pipeline

#### Drop redundant columns

* `encounter_id`, `patient_nbr` useless
* `weight`, `payer_code`, `medical_specialty` too many missing values
* `citoglipton`, `examide` only one value

In [5]:
columns_to_drop = [
    'encounter_id', 'patient_nbr', 'payer_code', 'medical_specialty',
    'citoglipton', 'examide'
]

#### Drop redundant rows

In [6]:
indices_to_drop = set()

In [7]:
indices_to_drop.update(list(df[df.diag_1.isna() & df.diag_2.isna() & df.diag_3.isna()].index))

In [8]:
indices_to_drop.update(list(df[~df.gender.isin(['Male', 'Female'])].index))

### Preprocessing using Pipeline

In [9]:
preprocessing_pipeline = Pipeline([
    ('cols_nan_filter', ColumnsNanFilter()),
    ('cols_filter', ColumnsFilter(columns_to_drop)),
    ('cols_values_diversity_filter', ColumnsValuesDiversityFilter(0.9)),
    ('rows_filter', RowsFilter(indices_to_drop))
])

In [13]:
preprocessing_pipeline.fit(df)
df_preprocessed = preprocessing_pipeline.transform(df)

ColumnsNanFilter transformation started.
ColumnsNanFilter transformation ended, took 0.03 seconds.
ColumnsFilter transformation started.
ColumnsFilter transformation ended, took 0.03 seconds.
ColumnsValuesDiversityFilter transformation started.
ColumnsValuesDiversityFilter transformation ended, took 0.01 seconds.
ColumnsNanFilter transformation started.
ColumnsNanFilter transformation ended, took 0.03 seconds.
ColumnsFilter transformation started.
ColumnsFilter transformation ended, took 0.03 seconds.
ColumnsValuesDiversityFilter transformation started.
ColumnsValuesDiversityFilter transformation ended, took 0.01 seconds.
RowsFilter transformation started.
RowsFilter transformation ended, took 0.02 seconds.


In [14]:
df_preprocessed.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'A1Cresult', 'metformin',
       'glipizide', 'glyburide', 'insulin', 'change', 'diabetesMed',
       'readmitted'],
      dtype='object')